In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib

Pre-processing and analyzing data (data was originally cleaned in excel)

In [ ]:
# Loading in pre-processed data set
df = pd.read_csv (r'/content/TrainOnMe-edited.csv')

# Encoded values in data set
# y = {0 : Shoogee, 1 : Bob, 2 : Jorg, 3 : Atsuto}
# x12 = {0 : False, 1 : True}

# Cleaning data and creating train/test sets
df = pd.read_csv (r'/content/TrainOnMe-edited.csv')
df = df.drop(df.tail(1).index) # Nan
df = df.drop(841) # Corrupted data point left after pre-processing
labels = df["y"].values
df = df.drop(columns=["Unnamed: 0","y"]) # Data cleaning
df = df.drop(columns=['x1', 'x3', 'x7', 'x12', 'x13']) # Feature selection

# Check balance of data set
test = [0,0,0,0]
for i in range(len(labels)):
 test[int(labels[i])] += 1
print(test)
# [290, 268, 340, 95]

# Basic oversampling, use data of class 3 3x extra.
X = []
Y = []
for i in range(len(labels)):
  # print(df.values[i])
  if int(labels[i]) == 3:
    X.append(df.values[i])
    X.append(df.values[i])
    Y.append(labels[i])
    Y.append(labels[i])
  X.append(df.values[i])
  Y.append(labels[i])
# print(len(X))  # 1183
# print(len(Y))  # 1183

# Division into train/test set
x_train = X[:900]
x_test = X[900:]
y_train = Y[:900]
y_test = Y[900:]

PCA

In [ ]:

df=(df - df.mean()) / df.std()
pca = PCA(n_components=13)
comps = pca.fit_transform(df)
plt.plot(comps)
loadings = pd.DataFrame(pca.components_.T,
columns=['PC%s' % _ for _ in range(len(df.columns))],
index=df.columns)
plt.plot(pca.explained_variance_ratio_)
plt.ylabel('Explained Variance')
plt.xlabel('Components')
plt.show()
print(np.argmax(comps[:,4]))
# Helped localize outlier : index 841

Feature selection

In [ ]:
# Features selection
sel = SelectFromModel(RandomForestClassifier(n_estimators = 1300),threshold="mean") # Feature selection
sel.fit(x_train, y_train)
features_to_rm = df.columns[np.where(sel.get_support() == False)]
print(features_to_rm)

Baseline (several classifiers were tested, RF performed best of those)

In [ ]:
clf = RandomForestClassifier(n_estimators=1300,random_state=0).fit(x_train, y_train)
clf.score(x_test, y_test)
# 0.618

Parameter search 1

In [ ]:

grid = {'max_depth': [1,3,5,10],
 'max_features': ['auto'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 6, 10,],
 'n_estimators' : [200,400,800,1200,2000]}

rf_gs = GridSearchCV(estimator = RandomForestClassifier(), param_grid=grid, cv=6,verbose=1)
rf_gs.fit(X,Y)
print(rf_gs.best_params_)
# {'max_depth': 5, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}

Parameter search 2

In [ ]:

grid = {'max_depth': [5,7],
 'max_features': ['auto'],
 'min_samples_leaf': [1],
 'min_samples_split': [8,10,20,100],
 'n_estimators' : [200,300]}

rf_gs = GridSearchCV(estimator = RandomForestClassifier(), param_grid=grid, cv=6,verbose=1)
rf_gs.fit(X,Y)
print(rf_gs.best_params_)
# {'max_depth': 7, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 300}

Training the classifier using found parameters

In [ ]:

rf = RandomForestClassifier(n_estimators=600, max_depth=7, min_samples_split=8,random_state=0)
scores = cross_val_score(rf, X, Y, cv=10)
print("%0.3f accuracy with a standard deviation of %0.3f" % (scores.mean(), scores.std()))
# 0.636 accuracy with a standard deviation of 0.053
# n_est chosen to 600 after some short testing
clf = rf.fit(X,Y)

Creating output file consisting of predicted labels according to formatting instructions

In [ ]:
ev = pd.read_csv (r'/content/EvaluateOnMe-edited.csv')
ev = ev.drop(columns=["Unnamed: 0"])
ev = ev.drop(columns=['x1', 'x3', 'x7', 'x12', 'x13'])
# print(ev)
evData = ev.values

predVal = clf.predict(evData)
print(predVal)

intToLabel = ["Shoogee", "Bob", "Jorg", "Atsuto"]
predLabels = []
for val in predVal:
  predLabels.append(intToLabel[int(val)])

print(predLabels)
f = open(r'/content/temp.txt', "w")
for label in predLabels:
  f.write(label+'\n')
f.close()